Traning, and Experiments with Feature Engineering and Hyperopt

In [1]:
# First experiment without including feature engineering

import pandas as pd
df_preprocessed = pd.read_csv('Datasets/ds_feature_eng.csv')
pd.set_option('display.max_columns', None)
df_preprocessed.head()

,efficiency,points,plus_minus,turnovers,blocks_per_min,turnovers_per_min,team_LAL,opponent_GSW,team_CHA,steals_per_min,team_MIA,steals,points_per_min,opponent_IND,opponent_CHA,opponent_DAL,blocks,position_SF,team_OKC,opponent_DEN,team_DAL,assists_per_min,team_BKN,team_CHI,opponent_LAL,efficiency_per_min,team_HOU,opponent_CLE,opponent_UTA,opponent_LAC,opponent_OKC,scoring_consistency,team_NOP,game_location_Home,age,assists,rebounds_per_min,team_GSW,usage_rate,team_MIN,opponent_MIL,opponent_NOP,age_bin_prime,three_pct,pm_bin_neutral,position_PG,team_PHI,opponent_NYK,opponent_ORL,minutes_played,team_SAC,opponent_DET,rest_days,fg_pct,team_TOR,impact_metric,ft_pct,team_DET,position_PF,team_CLE,rebounds,opponent_SAC,opponent_SAS,team_LAC,target
0,12,9,-3,2,0.081967,0.163934,0,0,0,0.081967,0,1,0.737705,0,0,0,1,0,0,0,0,0.000000,0,0,0,0.983606,0,0,0,0,0,37.918030,1,1,25,0,0.245902,0,0.901639,0,0,0,1,19.6,0,0,0,0,0,12.2,0,0,2,51.4,0,0.983606,79.9,0,0,0,3,0,0,0,0
1,37,15,1,0,0.064725,0.000000,0,0,0,0.032362,0,1,0.485437,0,0,0,2,0,0,0,0,0.161812,0,0,0,1.197411,0,0,0,0,0,19.902912,0,0,25,5,0.453074,0,0.647249,0,0,0,1,31.2,1,0,0,0,0,30.9,0,0,1,41.0,0,1.100324,68.1,0,0,0,14,0,0,0,1
2,16,11,-2,2,0.000000,0.105820,0,0,0,0.052910,0,1,0.582011,0,0,0,0,0,0,0,0,0.105820,0,0,0,0.846561,0,0,0,0,0,27.587300,0,0,30,2,0.211640,0,0.793651,0,0,1,0,28.0,0,0,0,0,0,18.9,0,0,2,47.4,0,0.899471,78.2,0,1,0,4,0,0,0,0
3,15,12,7,3,0.000000,0.150000,0,0,0,0.000000,0,0,0.600000,0,0,0,0,0,0,0,0,0.100000,0,0,0,0.750000,0,0,0,0,0,29.579999,0,0,27,2,0.200000,0,0.850000,0,0,0,1,45.8,0,0,0,0,0,20.0,0,0,2,49.3,0,0.900000,68.1,0,0,0,4,0,0,0,0
4,29,16,7,2,0.000000,0.071942,0,0,0,0.035971,0,1,0.575540,0,0,0,0,0,0,0,0,0.143885,0,0,0,1.043165,0,0,0,0,0,19.856114,0,0,29,4,0.359712,0,0.791367,0,0,0,0,22.5,0,0,1,0,1,27.8,0,0,1,34.5,0,1.079137,76.0,0,0,0,10,0,0,0,1


In [2]:
# Test split for the experiment without feature engineering
from sklearn.model_selection import train_test_split

# Defining features and target
X_1 = df_preprocessed.drop(columns='target')
y_1 = df_preprocessed['target']

# Test Split (20% of total data)
X_1_train_valid, X_1_test, y_1_train_valid, y_1_test = train_test_split(
    X_1, y_1, test_size=0.2, random_state=12345
)

# Validation split (25% of the remaining 80% → 20% of total)
X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(
    X_1_train_valid, y_1_train_valid, test_size=0.25, random_state=12345
)

print(
    f"Training set Rows: {X_1_train.shape[0]} - ",
    f"Validation set Rows: {X_1_valid.shape[0]} - ",
    f"Test set Rows: {X_1_test.shape[0]}"
)

Training set Rows: 5400 -  Validation set Rows: 1800 -  Test set Rows: 1800


Checking Class Balance in the Training Dataset:

In [3]:
# Assuming y_train is a Pandas Series
# The primary reason for checking the class balance is to prevent the model from being biased toward the majority class.
class_counts = y_1_train.value_counts()
print("class counts")
print(class_counts)

class_proportions = y_1_train.value_counts(normalize=True)
print("class Proportions")
print(class_proportions)

class counts
target
0    3212
1    2188
Name: count, dtype: int64
class Proportions
target
0    0.594815
1    0.405185
Name: proportion, dtype: float64


Class Balance Conclusion:
- The dataset has a ~60/40 split between the two classes.
- Is this imbalanced? Yes, it is imbalanced because the classes are not equal (50/50).
- Is it severely imbalanced? No. A severe imbalance is usually considered to be anything worse than 80/20 or 90/10.

Implications:
- It is manageable, but it still requires attention when predicting the minority class.
- Better to use the Precision, Recall, F1-Score, and ROC AUC. To check how well perform with the minotry class.
- Adjust class weighting: for XGB set the "scale_pos_weight" parameter. Starting value is the ratio of negative to positive samples 3212/2188 = 1.47. It tells the model to give the moniroty class 1.47 times the importance of the majority class.

Later conclusion:
- Conclusion about adding the balacing:
  - Recall improve: model is now much better at detecting the minority class (your positives).

MLflow Experiment Creation

In [4]:
# --- Create MLflow Experiment ---
import mlflow
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="http://127.0.0.1:5000/")

# Set up MLflow experiment
experiment_description = (
    "This is the WZ challenge about NBA player performance. "
    "This experiment contains an XGBoost model with hyperparameter tuning using Hyperopt."
)
experiment_tags = {
    "team": "data_science",
    "project_manager": "carlos_himura",
    "project_name": "nba_binary_classification",
    "mlflow.note.content": experiment_description,
}
experiment_name = "XGBoost_Classification_Feat_Eng"
xgboost_experiment = client.create_experiment(name = experiment_name, tags = experiment_tags)

search_experiment = client.search_experiments(
  filter_string="tags.`project_name` = 'nba_binary_classification'"
)
print("Searching by Experiment Tag - Project Name:")
print(search_experiment[0])

Searching by Experiment Tag - Project Name:
<Experiment: artifact_location='mlflow-artifacts:/982562299770504179', creation_time=1764714729591, experiment_id='982562299770504179', last_update_time=1764714729591, lifecycle_stage='active', name='XGBoost_Classification_Feat_Eng', tags={'mlflow.note.content': 'This is the WZ challenge about NBA player '
                        'performance. This experiment contains an XGBoost '
                        'model with hyperparameter tuning using Hyperopt.',
 'project_manager': 'carlos_himura',
 'project_name': 'nba_binary_classification',
 'team': 'data_science'}>


Model Training with Hyperopt and with Feature Engineering

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# NOTE: The ratio for scale_pos_weight is calculated from your class counts (3212 / 2188)
# This parameter is fixed here as it directly addresses the known data imbalance.
CLASS_WEIGHT_RATIO = 3212 / 2188 

# --- 1. Define the Search Space ---
# Defining the range of values that Hyperopt will test for each hyperparameter.
# Using the defaults as a guide to create the ranges.

search_space = {
    "n_estimators":     hp.quniform("n_estimators", 200, 1500, 50), # Number of trees
    "max_depth":        hp.quniform("max_depth", 3, 6, 1), # Maximum depth of each tree
    "learning_rate":    hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)), # Boosting learning rate
    "subsample":        hp.uniform("subsample", 0.8, 1.0), # % of rows sampled per tree
    "colsample_bytree": hp.uniform("colsample_bytree", 0.75, 1.0), # % of features sampled per tree
    "gamma":            hp.uniform("gamma", 0, 20), # Minimum loss reduction required to make a further partition on a leaf node of the tree
    "min_child_weight": hp.quniform("min_child_weight", 5, 20, 1), # It acts as a regularization term to prevent the model from learning relationships that are too specific to the training data
}

# --- 2. Define the objective function ---
def objective(params):

    # Convert integer params
    params["n_estimators"]     = int(params["n_estimators"])
    params["max_depth"]        = int(params["max_depth"])
    params["min_child_weight"] = int(params["min_child_weight"])

    model = XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss", # Evaluation metric
        random_state=12345, # Reproducibility
        tree_method="hist", # Fast + recommended
        scale_pos_weight=CLASS_WEIGHT_RATIO, #  BALANCE THE CLASSES
        **params
    )

    model.fit(
        X_1_train, y_1_train,
        eval_set=[(X_1_valid, y_1_valid)],
        verbose=False
    )

    # Predict validation
    y_val_pred = model.predict(X_1_valid) # Accuracy, F1, Precision, Recall, Confusion Matrix
    y_val_proba = model.predict_proba(X_1_valid)[:, 1] # ROC-AUC (metric for Hyperopt)

    val_roc = roc_auc_score(y_1_valid, y_val_proba)  # Use ROC-AUC to optimize

    return {"loss": -val_roc, "status": STATUS_OK}

# --- 3. Run Hyperopt Search ---
trials = Trials()

best_params = fmin(
    fn=objective, # Function to minimize
    space=search_space,
    algo=tpe.suggest, # Search algorithm (TPE = Tree-structured Parzen Estimator)
    max_evals=100, # Number of different parameter sets to try
    trials=trials, # Object to store all results
    rstate=np.random.default_rng(42)
)

# Convert integer hyperparameters returned by Hyperopt
best_params["n_estimators"]     = int(best_params["n_estimators"])
best_params["max_depth"]        = int(best_params["max_depth"])
best_params["min_child_weight"] = int(best_params["min_child_weight"])


print("\n Best Hyperopt Parameters:")
print(best_params)

# --- 4. TRAIN FINAL MODEL WITH BEST PARAMETERS ---

xgb_clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=12345,
    tree_method="hist",
    **best_params
)

xgb_clf.fit(
    X_1_train, y_1_train,
    eval_set=[(X_1_valid, y_1_valid)],
    verbose=False
)


# VALIDATION PREDICTIONS
y_val_pred = xgb_clf.predict(X_1_valid)
y_val_proba = xgb_clf.predict_proba(X_1_valid)[:, 1]

val_acc_2  = accuracy_score(y_1_valid, y_val_pred)
val_roc_2  = roc_auc_score(y_1_valid, y_val_proba)
val_prec_2 = precision_score(y_1_valid, y_val_pred)
val_rec_2  = recall_score(y_1_valid, y_val_pred)
val_f1_2   = f1_score(y_1_valid, y_val_pred)

print("\n📊 Validation Results:")
print(f"Accuracy: {val_acc_2:.4f}")
print(f"ROC AUC: {val_roc_2:.4f}")
print(f"Precision: {val_prec_2:.4f}")
print(f"Recall (Sensitivity): {val_rec_2:.4f}")
print(f"F1-Score: {val_f1_2:.4f}")

# Confusion Matrix (Validation)
val_cm_2 = confusion_matrix(y_1_valid, y_val_pred)
print("\n🔷 Validation Confusion Matrix:")
print("[[True_Pos False_Pos]")
print("[False_Neg True_Neg]]")
print(val_cm_2)

# TEST PREDICTIONS
y_test_pred = xgb_clf.predict(X_1_test)
y_test_proba = xgb_clf.predict_proba(X_1_test)[:, 1]

test_acc_2  = accuracy_score(y_1_test, y_test_pred)
test_roc_2  = roc_auc_score(y_1_test, y_test_proba)
test_prec_2 = precision_score(y_1_test, y_test_pred)
test_rec_2  = recall_score(y_1_test, y_test_pred)
test_f1_2   = f1_score(y_1_test, y_test_pred)

print("\n----------------------------------------")
print("📊 Test Results:")
print(f"Accuracy: {test_acc_2:.4f}")
print(f"ROC AUC: {test_roc_2:.4f}")
print(f"Precision: {test_prec_2:.4f}")
print(f"Recall (Sensitivity): {test_rec_2:.4f}")
print(f"F1-Score: {test_f1_2:.4f}")

# Confusion Matrix (Test)
test_cm_2 = confusion_matrix(y_1_test, y_test_pred)
print("\n🔷 Test Confusion Matrix:")
print("[[True_Pos False_Pos]")
print("[False_Neg True_Neg]]")
print(test_cm_2)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 100/100 [03:36<00:00,  2.17s/trial, best loss: -0.9325170707526028]

 Best Hyperopt Parameters:
{'colsample_bytree': np.float64(0.92790914912747), 'gamma': np.float64(11.223555562043511), 'learning_rate': np.float64(0.2576999175353665), 'max_depth': 4, 'min_child_weight': 15, 'n_estimators': 1400, 'subsample': np.float64(0.824988649268367)}

📊 Validation Results:
Accuracy: 0.8517
ROC AUC: 0.9324
Precision: 0.8202
Recall (Sensitivity): 0.8077
F1-Score: 0.8139

🔷 Validation Confusion Matrix:
[[True_Pos False_Pos]
[False_Neg True_Neg]]
[[949 128]
 [139 584]]

----------------------------------------
📊 Test Results:
Accuracy: 0.8500
ROC AUC: 0.9289
Precision: 0.8160
Recall (Sensitivity): 0.7852
F1-Score: 0.8003

🔷 Test Confusion Matrix:
[[True_Pos False_Pos]
[False_Neg True_Neg]]
[[989 122]
 [148 541]]


Logging the Differents MLflow Runs inside the Created Experiment After Hyperopt

In [11]:
# --- Logging the initial model to MLflow ---
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
import pandas as pd

# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Sets the current active experiment to the "XGBoost_Classification_Feat_Eng" experiment and returns the Experiment metadata
xgboost_experiment = mlflow.set_experiment("XGBoost_Classification_Feat_Eng")

# Define a run name for this iteration of training. If not set, a unique name will be auto-generated 
run_name = "XGB_Feat_run_v3"

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:

 # Log run tags
  mlflow.set_tag("model_type", "xgboost")
  mlflow.set_tag("feature_engineering", "yes")
  mlflow.set_tag("hyperopt", "yes")
  mlflow.set_tag("class_weighting", "yes")
  mlflow.set_tag("author", "carlos_himura")

 # FIX: Log the scale_pos_weight parameter explicitly
  mlflow.log_param("scale_pos_weight", CLASS_WEIGHT_RATIO) 

 # Log the parameters used for the model fit
  mlflow.log_params(best_params)

  # Log the metrics that were calculated during validation
  
  # Validation metrics
  mlflow.log_metric("val_accuracy", float(val_acc_2))
  mlflow.log_metric("val_roc_auc", float(val_roc_2))
  mlflow.log_metric("val_precision", float(val_prec_2))
  mlflow.log_metric("val_recall", float(val_rec_2))
  mlflow.log_metric("val_f1_score", float(val_f1_2))

  # Test metrics
  mlflow.log_metric("test_accuracy", float(test_acc_2))
  mlflow.log_metric("test_roc_auc", float(test_roc_2))
  mlflow.log_metric("test_precision", float(test_prec_2))
  mlflow.log_metric("test_recall", float(test_rec_2))
  mlflow.log_metric("test_f1_score", float(test_f1_2))

  # Validation Confusion Matrix as CSV
  val_cm_df = pd.DataFrame(val_cm_2)
  val_cm_df = pd.DataFrame(val_cm_2, 
                         index=["Pred_Pos_1 (TP-FP)", "Pred_Neg_0 (FN-TN)"], 
                         columns=["Act_Pos_1", "Act_Neg_0"])
  val_cm_df.to_csv("validation_confusion_matrix.csv")
  mlflow.log_artifact("validation_confusion_matrix.csv")

  # Test Confusion Matrix as CSV
  test_cm_df = pd.DataFrame(test_cm_2)
  test_cm_df = pd.DataFrame(test_cm_2,
                          index=["Pred_Pos_1 (TP-FP)", "Pred_Neg_0 (FN-TN)"],
                          columns=["Act_Pos_1", "Act_Neg_0"])
  test_cm_df.to_csv("test_confusion_matrix.csv")
  mlflow.log_artifact("test_confusion_matrix.csv")

  # Generate predictions for signature inference
  y_pred = xgb_clf.predict(X_1_test)

  # Create model signature and input example
  signature = infer_signature(X_1_test, y_pred)
  input_example = X_1_test.iloc[:5]

  # Log an instance of the trained model for later use
  mlflow.xgboost.log_model(
        xgb_clf, 
        name="xgb_clf",
        registered_model_name="XGB_Class_Feat_Eng", # If the model name doesn’t exist MLflow creates it. If it already exists MLflow automatically creates a new version
        input_example=input_example, # A small, representative example of the kind of input the model expects
        signature=signature # Model signature describing model input and output schema
        )
  

print(f"✅ Run '{run_name}' completed successfully.")

c:\Users\carlo\OneDrive\Documents\python_virtual_env\data_science_1\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Registered model 'XGB_Class_Feat_Eng' already exists. Creating a new version of this model...
2025/12/02 18:02:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB_Class_Feat_Eng, version 3
Created version '3' of model 'XGB_Class_Feat_Eng'.


🏃 View run XGB_Feat_run_v3 at: http://127.0.0.1:5000/#/experiments/982562299770504179/runs/f36f2bdf2ace450e9aea103fbce7c0b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/982562299770504179
✅ Run 'XGB_Feat_run_v3' completed successfully.


Metrics description 
- ROC-AUC: Measures the model’s ability to distinguish between classes
- Accuracy: Overall correctness of the model.
- Precision: Out of all predicted positives, how many were correct?
- Recall (Sensitivity): Out of all actual positives, how many were detected?
- F1 Score: Balance between precision and recall.

Metrics and Model Conclusion:

NBA Classification — What Metrics Should We Use to Choose the Best Model?
- The goal is to predict whether an NBA player would score over a certain threshold (binary classification).
- The 60/40 imbalance has been handled with class_weight.
- It should focus on metrics that are sensitive to both False Positives and False Negatives. Keeping in mind an accuracy of 65% might just mean it predicted the majority class correctly.
- Primary the ROC AUC: It evaluates the model's ability to distinguish between the two classes across all possible classification thresholds.
- Secondary F1-Score: is the harmonic mean of Precision and Recall. It provides a single number that requires the model to have high scores on both.
- Thrid Precision and Recall: Minimizing False Positives / Minimizing False Negatives.

Challenge especification: The dataset includes natural variability. Well-built models
typically achieve around 0.80 accuracy and 0.85–0.90 ROC-AUC without overfitting.

- Conclusion about adding the balacing:
  - Recall improve: model is now much better at detecting the minority class (your positives).


Selecting the Best Model, Parameters, and Metrics from MLflow

In [13]:
from mlflow import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("XGBoost_Classification_Feat_Eng")

# Fetch all runs for the experiment
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    # order_by=["metrics.test_roc_auc DESC"],  # Sort by best test ROC-AUC
    order_by=["metrics.test_roc_auc DESC", "metrics.test_accuracy DESC"],  # Sort by best test ROC-AUC and Accuracy
)

# Select the best run (first in the sorted list)
best_run = runs[0]
print(f"Best Run Name: {best_run.info.run_name}")
print(f"Best Run ID: {best_run.info.run_id}")

summary_data = {
    "Test Accuracy": [best_run.data.metrics.get("test_accuracy")],
    "Test ROC-AUC": [best_run.data.metrics.get("test_roc_auc")],
    "Test Precision": [best_run.data.metrics.get("test_precision")],
    "Test Recall": [best_run.data.metrics.get("test_recall")],
    "Test F1-Score": [best_run.data.metrics.get("test_f1_score")],
}

# Create a pandas DataFrame for a clean table
summary_df = pd.DataFrame(summary_data)

# Display neatly
print("\nBest Run Summary Metrics:")
display(summary_df.style.hide(axis="index"))

# Extract best run parameters
params_data = best_run.data.params

# Convert to a DataFrame
params_df = pd.DataFrame(list(params_data.items()), columns=["Parameter", "Value"])

# Display nicely without index
print("Best Run Parameters:")
display(params_df.style.hide(axis="index"))


Best Run Name: XGB_Feat_run_v2
Best Run ID: 638c59328a41400c8c8386e85267e6fc

Best Run Summary Metrics:


Test Accuracy,Test ROC-AUC,Test Precision,Test Recall,Test F1-Score
0.851667,0.929431,0.813988,0.793904,0.803821


Best Run Parameters:


Parameter,Value
colsample_bytree,0.9507579290048368
gamma,9.887094626323933
learning_rate,0.1371656701428185
max_depth,3
min_child_weight,10
n_estimators,1150
scale_pos_weight,1.4680073126142597
subsample,0.8702299683381548
